In [1]:
%pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 4.9 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("Data & Me API KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [3]:
import os
import sys
import asyncio
import nest_asyncio
from PIL import Image
import io
from pypdf import PdfReader
from kaggle_secrets import UserSecretsClient

# Patch asyncio to work inside Jupyter Notebooks
nest_asyncio.apply()

# --- ADK Imports ---
from google.adk.agents import LlmAgent
from google.adk.models import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [18]:
# --- CONFIGURATION ---
from typing import Literal, Optional, Dict
from google.adk.agents import LlmAgent
from google.adk.models import Gemini

MODEL_NAME = "gemini-2.5-flash-lite"

AGENCY_KNOWLEDGE_BASE = {
    "company_name": "Data & Me Solutions",
    "contact_email": "support@dataandmesolutions.com",
    "services": [
        {"id": 1, "name": "Software Development", "desc": "Custom web apps (Python/Node/React)", "price": "$150/hr"},
        {"id": 2, "name": "Website Development", "desc": "WordPress Build (Setup + Theme + 5 Pages)", "price": "Fixed $2,500"},
        {"id": 3, "name": "Data Analytics", "desc": "GA4 Setup + Looker Studio Dashboard", "price": "Fixed $1,200"},
        {"id": 4, "name": "Basic SEO", "desc": "On-page optimization & Sitemap setup", "price": "Fixed $800"},
        {"id": 5, "name": "Graphic Design", "desc": "Logo design & basic brand guidelines", "price": "Fixed $1,500"},
        {"id": 6, "name": "Mobile App Development", "desc": "iOS & Android Apps (Flutter/React Native)", "price": "$150/hr (Min. 100 hrs)"}
    ]
}

# --- EMAIL CONFIG & TEMPLATES ---

AGENCY_EMAIL_CONFIG = {
    # The address you actually send from
    "from_email": "support@dataandmesolutions.com",
    # Optional: central inbox where escalations go
    "to_email_default": "office@dataandmesolutions.com",
    # Optional SMTP-style config 
    "smtp_host": "smtp.yourprovider.com",
    "smtp_port": 587,
    "smtp_username": "support@dataandmesolutions.com",
    "smtp_password_env_var": "SUPPORT_EMAIL_PASSWORD",  # read from env
}

EMAIL_TEMPLATES = {
    "general_support": {
        "subject": "Support request from {customer_name}",
        "body": (
            "Hi Data & Me Solutions team,\n\n"
            "A customer has requested help that should be handled via email.\n\n"
            "Customer name: {customer_name}\n"
            "Customer email: {customer_email}\n"
            "Urgency: {urgency}\n\n"
            "Conversation summary from the AI assistant:\n"
            "{conversation_summary}\n\n"
            "Customer’s key question or request:\n"
            "{customer_request}\n\n"
            "Please reply to the customer from {from_email}.\n\n"
            "Best regards,\n"
            "Data & Me Solutions AI Assistant"
        ),
    },
    "proposal_followup": {
        "subject": "Website / Software project follow-up for {customer_name}",
        "body": (
            "Hi Data & Me Solutions team,\n\n"
            "The customer is interested in a project and would like to continue over email.\n\n"
            "Customer name: {customer_name}\n"
            "Customer email: {customer_email}\n"
            "Project context:\n"
            "{conversation_summary}\n\n"
            "Recommended next step:\n"
            "{customer_request}\n\n"
            "Please send a detailed proposal or follow-up from {from_email}.\n\n"
            "Best regards,\n"
            "Data & Me Solutions AI Assistant"
        ),
    },
}

import os
from typing import Literal, Optional, Dict

def send_support_email(
    customer_email: str,
    customer_name: str,
    conversation_summary: str,
    customer_request: str,
    urgency: Literal["low", "medium", "high"] = "medium",
    template_key: str = "general_support",
    override_to_email: Optional[str] = None,
) -> Dict[str, str]:
    """
    Drafts and (optionally) sends an escalation email from Data & Me Solutions
    when chat support is not enough or the user requests email communication.

    Args:
        customer_email (str): The customer's email address.
        customer_name (str): The customer's name (or 'Customer' if unknown).
        conversation_summary (str): Short summary of the chat so far.
        customer_request (str): The main problem / request to highlight.
        urgency (str): 'low', 'medium', or 'high'.
        template_key (str): Which email template to use.
        override_to_email (str, optional): If set, send to this internal
            address instead of the default support inbox.

    Returns:
        dict: {
            "status": "sent" or "preview_only" or "error",
            "to_email": "...",
            "from_email": "...",
            "subject": "...",
            "body": "...",
            "error_message": "..." (only if status == "error")
        }
    """
    template = EMAIL_TEMPLATES.get(template_key, EMAIL_TEMPLATES["general_support"])

    from_email = AGENCY_EMAIL_CONFIG["from_email"]
    to_email = override_to_email or AGENCY_EMAIL_CONFIG["to_email_default"]

    subject = template["subject"].format(
        customer_name=customer_name or "Customer",
        customer_email=customer_email or "Not provided",
    )

    body = template["body"].format(
        customer_name=customer_name or "Customer",
        customer_email=customer_email or "Not provided",
        urgency=urgency,
        conversation_summary=conversation_summary,
        customer_request=customer_request,
        from_email=from_email,
    )

    # 🔐 Real sending is environment specific – this is a safe placeholder.
    # Uncomment & adapt this section with smtplib or your email provider's API.
    #
    # import smtplib
    # from email.message import EmailMessage
    #
    # try:
    #     msg = EmailMessage()
    #     msg["From"] = from_email
    #     msg["To"] = to_email
    #     msg["Subject"] = subject
    #     msg.set_content(body)
    #
    #     password = os.getenv(AGENCY_EMAIL_CONFIG["smtp_password_env_var"])
    #     with smtplib.SMTP(AGENCY_EMAIL_CONFIG["smtp_host"], AGENCY_EMAIL_CONFIG["smtp_port"]) as server:
    #         server.starttls()
    #         server.login(AGENCY_EMAIL_CONFIG["smtp_username"], password)
    #         server.send_message(msg)
    #
    #     return {
    #         "status": "sent",
    #         "to_email": to_email,
    #         "from_email": from_email,
    #         "subject": subject,
    #         "body": body,
    #     }
    # except Exception as e:
    #     return {
    #         "status": "error",
    #         "error_message": str(e),
    #         "to_email": to_email,
    #         "from_email": from_email,
    #         "subject": subject,
    #         "body": body,
    #     }

    # Default: just return a preview so you can inspect it in logs / UI
    return {
        "status": "preview_only",
        "to_email": to_email,
        "from_email": from_email,
        "subject": subject,
        "body": body,
    }

# ==========================================
# 2. SYSTEM INSTRUCTIONS (The Logic Layer)
# ==========================================
AGENCY_SYSTEM_INSTRUCTION = f"""
You are the official AI Agent for "{AGENCY_KNOWLEDGE_BASE['company_name']}".

### PRIME DIRECTIVE (SECURITY & GROUNDING):
1.  **SCOPE CHECK:** You can ONLY discuss the services listed in the "AGENCY KNOWLEDGE BASE".
2.  **OUT OF SCOPE / UNKNOWN:** If a user asks for a service not listed (e.g., Video Editing, Cloud Hosting) or asks a question you cannot answer, reply EXACTLY:
    *"I am not trained for this specific request. If you have any questions about our services and solutions please let me know here, or reach out to us at {AGENCY_KNOWLEDGE_BASE['contact_email']} with your requirements."*
3.  **NO HALLUCINATIONS:** Do not invent prices or services.

### AGENCY KNOWLEDGE BASE:
{AGENCY_KNOWLEDGE_BASE['services']}

### OPERATING MODES & WORKFLOWS:

**MODE A: TIER-1 CUSTOMER SUPPORT (Troubleshooter)**
* **Trigger:** User reports a bug, error (e.g., 403, 404), or site issue.
* **Step 1 (First Response - The Investigation):** * **Empathize:** Start by acknowledging the frustration (e.g., "I'm sorry to hear your site is having issues...").
    * **Guide:** Ask them to perform a preliminary check. Suggest checking their **Backend Dashboard** for alerts or disabling recent **Plugins** that might be causing conflicts.
    * **Collect Evidence:** Ask them to share specific error messages or a screenshot if possible.
* **Step 2 (Escalation - The Ticket):** * **Condition:** ONLY proceed to this step if the user replies that they "don't know how", "can't access the dashboard", or "the fix didn't work".
    * **Action:** Log the ticket.
    * **Response:** *"I understand. Since that didn't resolve it, I have logged a priority ticket for you (Ticket ID: #T-GEN-001). Our technical team has been notified and will contact you shortly."*

**MODE B: SALES & LEAD CAPTURE (The Closer)**
* **Trigger:** User asks about services ("Do you do online stores?", "I need an app").
* **Action:**
    1.  **Confirm & Pitch:** "Yes, we handle that! [Describe the matching service from Knowledge Base]."
    2.  **The Specific Closing:** You MUST end with this exact question:
        *"Please let me know if you need a **formal proposal** with an estimated quote, or if you have specific questions and would like to **schedule a discovery call** with our team?"*

**MODE B-2: CALL SCHEDULING (Booking)**
* **Trigger:** User says "I want a call", "Schedule a meeting", or "Talk to the team".
* **Step 1:** Reply: *"That sounds great. Please share your **Email Address** and your **Preferred Date/Time** so I can check our Sales Team's availability."*
* **Step 2:** (Once user provides Email/Time):
    * **Action:** Simulate a calendar booking.
    * **Response:** *"Perfect. I have scheduled a call for [User Time] and sent a calendar invite to [User Email]. We look forward to speaking with you!"*

**MODE C: PROPOSAL GENERATOR**
* **Trigger:** User explicitly asks for a "quote", "price", or "proposal" for a valid service.
* **Action:** Generate a Markdown proposal using ONLY knowledge base prices.

MODE D – EMAIL ESCALATION & HANDOVER

You MUST consider using the `send_support_email` tool in the following situations:

1. The user explicitly asks to:
   - "send me an email"
   - "continue this over email"
   - "can someone email me?"
   - or provides their email and asks for follow-up.

2. You have tried your best to help using the knowledge base and reasoning,
   but the user is still:
   - confused,
   - unhappy with the answer,
   - or the request clearly requires a human to step in
     (for example: complex scoping, billing disputes, custom quotes).

3. The conversation reaches a point where a **formal proposal**, **detailed quote**,
   or **manual investigation** is required (e.g. checking logs, analytics, custom integrations).

When you decide to escalate:

- FIRST, clearly explain to the user that you will escalate to the Data & Me Solutions team by email.
- ASK for:
  - their email address (if not already provided),
  - their name (optional but preferred),
  - any extra context they want the team to see.

Then call the `send_support_email` tool with:
- `customer_email`: the email provided by the user (or blank if unknown).
- `customer_name`: the name they provide (or "Customer" if unknown).
- `conversation_summary`: a short summary of what has been discussed so far,
  focusing on what the user needs.
- `customer_request`: a single, clear sentence capturing what they are asking for
  (e.g., "Customer wants a detailed quote for a WordPress site with booking system").
- `urgency`: "low", "medium", or "high" based on the conversation.
- `template_key`:
  - Use "proposal_followup" if they are asking for a project proposal or quote.
  - Otherwise use "general_support".

After the tool call:
- Tell the user that the email has been prepared / sent to the support team
  using the official support address from the knowledge base.
- Confirm what they should expect next (for example: a human reply within 1–2 business days).
"""

# --- AGENT CREATION ---

Data_and_Me_Solutions_Agent = LlmAgent(
    name="data_and_me_solutions_agent",
    model=Gemini(model=MODEL_NAME),
    instruction=AGENCY_SYSTEM_INSTRUCTION,
    # MCP function tool auto-converted to a schema-based tool
    tools=[send_support_email],
)

print("✅ Agent 'Data & Me Solutions Agent' is ready to work.")

# ADK CLI looks for this variable
root_agent = Data_and_Me_Solutions_Agent

✅ Agent 'Data & Me Solutions Agent' is ready to work.


In [5]:
!pip install google-genai fastapi uvicorn pypdf xhtml2pdf markdown python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.7/470.7 kB 12.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 83.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.7 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [20]:
import asyncio
from google.genai.types import Content, Part
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

APP_NAME = "Data_and_Me_Solutions_Agent"
USER_ID = "demo_user"
SESSION_ID = "demo_session_1"

# 1) In-memory session store
session_service = InMemorySessionService()

# 2) Create the session (async)
async def init_session():
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )

# Instead of 'run_until_complete', we just await it directly
await init_session() 

# 3) Create the Runner
runner = Runner(
    agent=Data_and_Me_Solutions_Agent,
    app_name=APP_NAME,
    session_service=session_service,
)

print("✅ Runner initialized with in-memory session.")

✅ Runner initialized with in-memory session.


In [21]:
import asyncio
from IPython.display import Markdown, display
from google.genai.types import Content, Part

# --- 2. RE-INITIALIZE THE AGENT ---
Data_and_Me_Solutions_Agent = LlmAgent(
    name="data_and_me_solutions_agent",
    model=Gemini(model=MODEL_NAME),
    instruction=AGENCY_SYSTEM_INSTRUCTION
)

runner = Runner(
    agent=Data_and_Me_Solutions_Agent,
    app_name=APP_NAME,
    session_service=session_service,
)

# --- 3. INPUT: TEST YOUR AGENT HERE ---

# SCENARIO 1: SUPPORT (Active)
#client_request_text = "HEY! My website is showing a 404 error and I'm losing customers. Fix this now!"

# SCENARIO 2: SALES (Commented out)
client_request_text = "Hi, I'm thinking about starting a clothing brand. Do you guys handle online stores? What platform do you use?"

# SCENARIO 3: PROPOSAL (Commented out)
#client_request_text = """Can you tell what is going on in the London"""

# Wrapper for input
user_message = Content(role="user", parts=[Part(text=client_request_text)])

print(f"⏳ Processing Request: '{client_request_text[:50]}...'")

try:
    response_text = ""
    
    # Run the Agent
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=user_message
    ):
        if hasattr(event, 'text') and event.text:
            response_text += event.text
        elif hasattr(event, 'content') and hasattr(event.content, 'parts'):
            for part in event.content.parts:
                if hasattr(part, 'text'):
                    response_text += part.text

    # Display Output
    if response_text.strip():
        print("\n✅ Response Received:\n")
        print("-" * 60)
        display(Markdown(response_text)) 
        print("-" * 60)
    else:
        print("❌ Error: The agent returned an empty response.")

except Exception as e:
    print(f"❌ Error during execution: {e}")

⏳ Processing Request: 'Hi, I'm thinking about starting a clothing brand. ...'

✅ Response Received:

------------------------------------------------------------


Yes, we can help you build an online store for your clothing brand! While we don't have a specific "online store" service listed, we can leverage our **Website Development** service using WordPress. WordPress is a very popular and versatile platform that is excellent for e-commerce.

Please let me know if you need a **formal proposal** with an estimated quote, or if you have specific questions and would like to **schedule a discovery call** with our team?

------------------------------------------------------------


In [ ]:
!adk create Data_and_Me_Solutions_Agent --model gemini-2.5-flash-lite --api_key "Data & Me API KEY"

Non-empty folder already exist: '/kaggle/working/Data_and_Me_Solutions_Agent'
Override existing content? [y/N]: 

In [22]:
from jupyter_server.serverapp import list_running_servers

# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"
    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")
    baseURL = servers[0]["base_url"]
    return f"{PROXY_HOST}{baseURL}proxy/{ADK_PORT}/"

In [24]:
url_prefix = get_adk_proxy_url()

In [23]:
!adk web --url_prefix {url_prefix}

^C
Traceback (most recent call last):
  File "/usr/local/bin/adk", line 4, in <module>
    from google.adk.cli import main
  File "/usr/local/lib/python3.11/dist-packages/google/adk/__init__.py", line 16, in <module>
    from .agents.llm_agent import Agent
  File "/usr/local/lib/python3.11/dist-packages/google/adk/agents/__init__.py", line 18, in <module>
    from .base_agent import BaseAgent
  File "/usr/local/lib/python3.11/dist-packages/google/adk/agents/base_agent.py", line 40, in <module>
    from ..events.event import Event
  File "/usr/local/lib/python3.11/dist-packages/google/adk/events/__init__.py", line 15, in <module>
    from .event import Event
  File "/usr/local/lib/python3.11/dist-packages/google/adk/events/event.py", line 26, in <module>
    from ..models.llm_response import LlmResponse
  File "/usr/local/lib/python3.11/dist-packages/google/adk/models/__init__.py", line 19, in <module>
    from .gemma_llm import Gemma
  File "/usr/local/lib/python3.11/dist-packages/goog

In [13]:
print(get_adk_proxy_url())

https://kkb-production.jupyter-proxy.kaggle.net/k/283428086/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..ia1BvLEjYuT5Nxc6z_G1nw.kvIwh4o_BTa1CHcmwdvBx1So7htxyXfY33ylX9-9wGaUc92wurJqw0UZd4G9jEA1Xy7sb9OvDLAGWs0yNEDcptH8u0Z1tSdoXmfAJJ6iBclTn_63LnNP0_JLgJyK-ez1pd2alhgZRuwPn2NnCi7ASjbq-KObtZvdKP5y-gwalrKRfBwyrEopDjGaxeNRvY7kxMjy91uHUBtJ3rklAsULgD25d6mSqKYwnszsNKVcMr9gv9iSi-FcP3d_zWVANE_l.F3icMTFgemk0678ymUGPkw/proxy/proxy/8000/
